In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
pip install google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.9/319.9 kB 8.1 MB/s eta 0:00:00:00:01
  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0
  Attempting uninstall: cachetools
    Found existing installation: cachetools 6.2.1
    Uninstalling cachetools-6.2.1:
      Successfully uninstalled cachetools-6.2.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
google-cloud-translate 3.12.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.29.5 which is incompatible.
ray 2.51.1 requires click!=8.3.0,>=7.0, but you have click 8.3.0 which is incompatible.
bigframes 2.12.0 requires rich<14

In [28]:
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )


✅ Gemini API key setup complete.


In [10]:
from google.adk.agents import Agent, SequentialAgent, ParallelAgent, LoopAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import AgentTool, FunctionTool, google_search
from google.genai import types

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


In [13]:
retry_config=types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504], # Retry on these HTTP errors
)

In [14]:
error_check_agent=Agent(
    name="errorCheckAgent",
    model=Gemini(
        model="gemini-2.5-pro",
        retry_options=retry_config
    ),
    instruction="""You are a error finding agent.Your only job is to use  check if there are any errors
    in code . If there are errors then tell user the errors and if there aren't any errors then just tell "NO error ,You can run this code".
    """,
    output_key="error_report",
    
)
print("error_check_agent created.")

error_check_agent created.


In [21]:
def safe_run_code(code: str):
    try:
        exec(code, {})
        return "Code executed successfully"
    except Exception as e:
        return f"Runtime Error: {str(e)}"


In [20]:
run_code_tool = FunctionTool(safe_run_code)


In [22]:
run_code_agent = Agent(
    name="RunCodeAgent",
    model=Gemini(
        model="gemini-2.5-pro",
        retry_options=retry_config
    ),
    instruction="""
You are a code execution agent.

Error Check Result: {error_report}

Your only job:
- If the error_check_agent output is EXACTLY "NO error, You can run this code",
  then you MUST call the run_code_tool and execute the code.
- If {error_report} contains any errors, you MUST NOT run the code.

Show only the final execution result.
""",
    tools=[run_code_tool],
    output_key="execution_result"
)

print("✅ run_code_agent created.")


✅ run_code_agent created.


In [23]:
root_agent = Agent(
    name="Code_Runner",
    model=Gemini(
        model="gemini-2.5-pro",
        retry_options=retry_config
    ),
    instruction="""
You are a code runner agent. Your job is to first detect errors in the code, and then run the code if it has no errors.

Steps:
1. First, you MUST call the `ErrorCheckAgent` tool to check for errors in the code.
2. After receiving the result, if the output is EXACTLY "NO error, You can run this code", you MUST call the `RunCodeAgent` tool to run the code.
3. Finally, provide the final execution result to the user.
""",
    tools=[
        AgentTool(error_check_agent),
        AgentTool(run_code_agent)
    ],
)

print("root_agent created")


root_agent created


In [36]:
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug(
    "prin('hello world')"
)



 ### Created new session: debug_session_id

User > prin('hello world')


Code_Runner > There is an error in your code.

**Error:** `NameError: name 'prin' is not defined`

**Reason:** The function to print output in Python is spelled `print`, not `prin`.

**Correction:**
```python
print('hello world')
```
